In [1]:
import pandas as pd
import re
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
at_least_1_office = db.companies.find({"offices":{"$not":{"$size":0}}})


In [3]:
df = pd.DataFrame(at_least_1_office)

In [4]:
def getFirst(data):

    data = data['offices']

    principal = None

    if data[0]['latitude'] and data[0]['longitude']:

        principal = {

            "type":"Point",

            "coordinates":[data[0]['longitude'], data[0]['latitude']]

        }

    return {

        "totalOffices": len(data),

        "lat": data[0]['latitude'],

        "lng": data[0]['longitude'],

        "oficina_principal": principal

    }

Geoloc = df[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [5]:
df.drop(['_id', 'acquisition','created_at', 'crunchbase_url', 'ipo', 'homepage_url', 'image', 'partners', 'providerships', 'updated_at',  'permalink', 'products', 'external_links', 'acquisitions', 'alias_list', 'blog_feed_url', 'blog_url', 'category_code', 'deadpooled_day', 'deadpooled_month', 'deadpooled_url', 'email_address', 'founded_day', 'founded_month', 'phone_number', 'screenshots', 'tag_list', 'twitter_username', 'video_embeds', 'relationships', 'milestones', 'investments', 'overview', 'funding_rounds', 'competitions'] , axis=1, inplace=True)

In [6]:
df= df[['description', 'founded_year', 'name', 'total_money_raised', 'number_of_employees', 'deadpooled_year']].fillna('Unknown')

In [7]:
DataBase = pd.concat([df, Geoloc], axis=1)

In [8]:
def showmedamoney (m):
    
    m=str(m)
    
    floatmoney = float((re.findall('\d+|\d+.?\d+', m))[0])
    
    if m.startswith('C$'):x = floatmoney*1.31
    elif m.startswith('€'):x = floatmoney*0.89
    elif m.startswith('$'):x = floatmoney
    elif m.startswith('£'):x = floatmoney*0.80
    elif m.startswith('¥'):x = floatmoney*108.96
    else:
        x=floatmoney
        
    if m.endswith('M'):
        x = x*1000000
        return x
    elif m.endswith('B'):
        x = x*1000000000
        return x
    elif m.endswith('k'):
        x = x*1000
        return x
    else:
        return x

In [9]:
DataBase['total_money_raised']= DataBase['total_money_raised'].apply(showmedamoney)

In [10]:
def activity (m):
    m = str(m)
    
    if re.search("s*(design)|(Design)\s*", m):
        return m
    elif re.search("s*(Video)|(video)\s*", m):
        return m
    elif re.search("s*(Game)|(game)\s*", m):
        return m
    elif re.search("s*(Gaming)|(gaming)\s*", m):
        return m
    
    else:
        return 'OTHERS'



In [11]:
DataBase['description']= DataBase['description'].apply(activity)

In [12]:
DataBase=DataBase[DataBase['description']!='OTHERS']

In [13]:
DataBase['total_money_raised'] = DataBase['total_money_raised'].fillna(0.0).astype(int)

In [14]:
DataBase=DataBase[DataBase['deadpooled_year']=='Unknown']

In [ ]:
DataBase